In [17]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import tqdm

In [2]:
# Utiliser cuda pour augmenter la rapidité d'exécution
# GPU/graphics driver supports a particular version CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
# Path for training and testing directory
train_path = 'content/OCT2017/train/'
test_path = 'content/OCT2017/test/'

img_width, img_height = 150, 150
batch_size = 32

In [4]:
# Preprocessing
transformer = transforms.Compose([
    transforms.Resize((img_width, img_height)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # 0-1 to [-1, 1], formula (x-mean)/std
])

In [5]:
# Dataloader
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size = batch_size,
    shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    batch_size = batch_size,
    shuffle = True
)

In [6]:
# Categories
root = pathlib.Path(train_path)
classes = [file.name.split('/')[-1] for file in root.iterdir()]
classes

['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [8]:
# CNN Network
class ConvNet(nn.Module):
    def __init__(self, num_classes = len(classes)):
        super(ConvNet,self).__init__()
        
        # Input shape= (256,3,150,150)
        self.conv1 = nn.Conv2d(
            in_channels = 3, 
            out_channels = 12, 
            kernel_size = 3, 
            stride = 1, 
            padding = 1
        )
        # Shape = (256,12,150,150)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1 = nn.ReLU()
        
        # Reduce the image size by factor of 2
        # Shape = (256,12,75,75)
        self.pool = nn.MaxPool2d(kernel_size=2)
        
        # Shape = (256,20,75,75)
        self.conv2 = nn.Conv2d(
            in_channels = 12,
            out_channels = 20,
            kernel_size = 3,
            stride = 1,
            padding = 1
        )
        self.relu2 = nn.ReLU()
        
        # Shape = (256,32,75,75)
        self.conv3 = nn.Conv2d(
            in_channels = 20,
            out_channels = 32,
            kernel_size = 3,
            stride = 1,
            padding = 1
        )
        self.bn3 = nn.BatchNorm2d(num_features = 32)
        self.relu3 = nn.ReLU()
        
        self.fc = nn.Linear(
            in_features = 75 * 75 * 32, 
            out_features = num_classes
        )
        
    # Feed forward function
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
             
            
        # Above output will be in matrix form, with shape (256,32,75,75) 
        output = output.view(-1, 32*75*75)
        output = self.fc(output)
            
        return output 

In [9]:
model = ConvNet(num_classes = len(classes)).to(device)

In [10]:
# Optimizer and loss function
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)
loss_function = nn.CrossEntropyLoss()

In [11]:
num_epochs = 10

In [12]:
# Calculating the size of training and testing images
train_count = len(glob.glob(train_path + '**/*.jpeg'))
test_count = len(glob.glob(test_path + '**/*.jpeg'))
print(train_count, test_count)

83484 968


In [16]:
len(train_loader)

2609

In [23]:
# Model training and saving best model
best_accuracy = 0.0

for epoch in range(num_epochs):
    # Evaluation and training on training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data, 1)
        
        train_accuracy += int(torch.sum(prediction == labels.data))        
        
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    print(f'Train Loss: {train_loss} Train Accuracy: {train_accuracy}')
    
    # Evaluation on testing dataset
    model.eval()
    test_accuracy = 0.0
    
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy) +' Test Accuracy: '+str(test_accuracy))
    
    # Save the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Train Loss: 0.43475809693336487 Train Accuracy: 0.8465454458339322
Epoch: 0 Train Loss: tensor(0.4348) Train Accuracy: 0.8465454458339322 Test Accuracy: 0.9028925619834711
Train Loss: 0.3414214253425598 Train Accuracy: 0.8814982511618993
Epoch: 1 Train Loss: tensor(0.3414) Train Accuracy: 0.8814982511618993 Test Accuracy: 0.9411157024793388
Train Loss: 0.2770894467830658 Train Accuracy: 0.9044846916774472
Epoch: 2 Train Loss: tensor(0.2771) Train Accuracy: 0.9044846916774472 Test Accuracy: 0.9597107438016529
Train Loss: 0.23658722639083862 Train Accuracy: 0.9195534473671602
Epoch: 3 Train Loss: tensor(0.2366) Train Accuracy: 0.9195534473671602 Test Accuracy: 0.9483471074380165
Train Loss: 0.2134728580713272 Train Accuracy: 0.9275070672224618
Epoch: 4 Train Loss: tensor(0.2135) Train Accuracy: 0.9275070672224618 Test Accuracy: 0.9183884297520661
Train Loss: 0.18720652163028717 Train Accuracy: 0.9372095251784773
Epoch: 5 Train Loss: tensor(0.1872) Train Accuracy: 0.9372095251784773 Test 